# AWS Cross-Account Resource Sharing Notebook
This notebook provides examples how to configure IAM Roles and Policies between a **Shared (Trusting) Account** which contains resources being made accessible to a **Trusted Account**.  It contains scripts to query existing configurations to discover what is already in place.   This workbook also contains templates based on a set of parameters common to both the Shared and Trusted accounts to support creating new configurations from scratch.  Cross-account configurations are not difficult, but they are detailed.  And several approaches yield similar results.  The goal this workbook is to promote standardization of Cross-account configurations.


In [4]:
import boto3
import json
import logging

""" Import Python Modules """
import sys
sys.path.insert( 0, './python') 

# load common code module for Cross Account access
import iam_cross_account as xa

import importlib
print(importlib.reload(xa))

help(xa)

<module 'iam_cross_account' from 'c:\\Users\\richa\\Code\\cross-account-resource-sharing\\iam_cross_account.py'>
Help on module iam_cross_account:

NAME
    iam_cross_account - Module to support Cross Account Roles and Policies between a Shared Account and a Trusted Account

CLASSES
    builtins.object
        AwsAccount
    
    class AwsAccount(builtins.object)
     |  AwsAccount(partition, region, account_id, profile_name)
     |  
     |  Class that represents an IAM Account with methods to manage Roles and Policies
     |  
     |  # adapted from https://github.com/awsdocs/aws-doc-sdk-examples/blob/main/python/example_code/iam
     |  # Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
     |  # SPDX-License-Identifier: Apache-2.0
     |  
     |  Methods defined here:
     |  
     |  __init__(self, partition, region, account_id, profile_name)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  attach_policy(self, role_name

In [5]:
# Define common parameters spec'd in both Shared and Trusted accounts
xacct_parms = {
    'Partition' : 'aws' ,                   # [aws|aws-us-gov]
    'SharedRegion' : 'us-east-2',
    'SharedAcct' : '442483223120', 
    'SharedKeyId' : 'daab-lab-kms-s3',      # uuid of KMS key,
    'SharedBucket' : 'daab-lab-smpl-main-landing-pad',
    'SharedFolder' : 'TOP',
    'SharedPrefix' : 'DAAB-LAB-RAM',        # $SharedSys-$Env-$TrustedSys
    'TrustedRegion' : 'us-east-2',
    'TrustedAcct' : '289755104220', 
    'TrustedPrefix' : 'RAM-LAB-DAAB',       # $TrustedSys-$Env-$SharedSys
}

# Create Object to work with Shared Account (wherein resources reside)
shared_acct = xa.AwsAccount (
    profile_name = 'ram_daab_lab',  # from ~/.aws/credentials file (ToDo: pass/set session)
    partition = xacct_parms['Partition'], 
    region =    xacct_parms['SharedRegion'],
    account_id= xacct_parms['SharedAcct'],
)
# Create Object to work with Trusted Account (to access shared resources)
trusted_acct = xa.AwsAccount (
    profile_name = 'ram_fh',        # from ~/.aws/credentials file (ToDo: pass/set session)
    partition = xacct_parms['Partition'], 
    region =    xacct_parms['TrustedRegion'],
    account_id= xacct_parms['TrustedAcct'],
)

xacct_parms

{'Partition': 'aws',
 'SharedRegion': 'us-east-2',
 'SharedAcct': '442483223120',
 'SharedKeyId': 'daab-lab-kms-s3',
 'SharedBucket': 'daab-lab-smpl-main-landing-pad',
 'SharedFolder': 'TOP',
 'SharedPrefix': 'DAAB-LAB-RAM',
 'TrustedRegion': 'us-east-2',
 'TrustedAcct': '289755104220',
 'TrustedPrefix': 'RAM-LAB-DAAB'}

#### Scenario 1 -- Query Existing Existing Role & Policy Configurations into a JSON document
An IAM Role, which specifies [Trust Policy](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_terms-and-concepts.html#term_trust-policy), grants permissions thru a combination of [Attached (Managed) Policies and Inline Policies](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_managed-vs-inline.html).  Managed Policies are generally preferred, especially if the same set of permissions will be granted to multiple Roles and/or Users.

On the IAM Console, reviewing a Role involves paging around thru several tabs and links.  This makes it inconvenient to document (with a mess a screenshots), and difficult to review during troubleshooting sessions.

The **query_role_and_policy_config(role_name)** method retrieves all the relevant Role and Policy information into a common JSON document, which can be written to a file for purposes of 'as-is' documentation and sharing for review.


In [6]:
# Query Existing Role & Policy Configurations into a JSON document
# e.g.,
# ... from the Trusted Account ...
role_name = 'ram-lab-CrossAccountLandingPadRole'
outfile = f'{role_name}-trusted_config.json'

trusted_config = trusted_acct.query_role_and_policy_config ( role_name )
with open(outfile, 'w') as f:
    f.write( json.dumps(trusted_config, indent=2) )

print(f"Wrote {outfile} from Account {trusted_acct.account_id}.")

# ... from the Shared Account ...
role_name = 'daab-lab-CrossAccountLandingPadRole'
outfile = f'{role_name}-shared_config.json'

shared_config = shared_acct.query_role_and_policy_config ( role_name )
with open(outfile, 'w') as f:
    f.write( json.dumps(shared_config, indent=2) )

print(f"Wrote {outfile} from Account {shared_acct.account_id}.")


Wrote ram-lab-CrossAccountLandingPadRole-trusted_config.json from Account 289755104220.
Wrote daab-lab-CrossAccountLandingPadRole-shared_config.json from Account 442483223120.


#### Scenario 2 -- Generate Cross Account Configurations from Templates 

Cross-Account configurations involve setting up the same parameters in both the Trusted and Sharing accounts.  We can use standardized templates for these configurations with 'placeholders' to represent common parameters.  

In [7]:
# Search and Replace JSON Template '$Placeholders' with Common Parameters
import  json

def replace_keyvals( parms, in_string ):
    for key, val in parms.items():
        print(key,val)
        in_string = in_string.replace( f"${key}", val)

    return in_string

#e.g.,
in_template = "$SharedPrefix-CrossAccountLandingPadRole-shared.json"
with open( in_template , "r") as f:
    in_buf = f.read()

shared_config = json.loads(replace_keyvals( xacct_parms, in_buf ))
shared_config['CrossAccountParms'] = xacct_parms
shared_config['FromTemplate'] = in_template

out_config = f"{shared_config['RoleName']}.json"
with open( out_config, "w") as f:
    f.write( json.dumps(shared_config, indent=2) )

print( f"Created '{out_config}' from '{in_template}'")


Partition aws
SharedRegion us-east-2
SharedAcct 442483223120
SharedKeyId daab-lab-kms-s3
SharedBucket daab-lab-smpl-main-landing-pad
SharedFolder TOP
SharedPrefix DAAB-LAB-RAM
TrustedRegion us-east-2
TrustedAcct 289755104220
TrustedPrefix RAM-LAB-DAAB
Created 'DAAB-LAB-RAM-CrossAccountLandingPadRole.json' from '$SharedPrefix-CrossAccountLandingPadRole-shared.json'


#### Scenario 3 -- Create/Update IAM Roles and Policies from JSON Config Doc

The **put_role_and_policy_config (config_doc)** method actually applies settings from JSON Config Docs to IAM Roles and Policies.  
 
JSON Config Docs specified in this project can be used to cut and paste into Terraform or CloudFormation scripts.  Or emailed to admins of other accounts.  New configurations might come from an edited copy of existing settings or get generated from templates and parameters.  


In [9]:
# Put IAM Role with Attached and/or Inline Policies from a JSON doc
# (e.g., to apply manual edits)
config_filename = "DAAB-LAB-RAM-CrossAccountLandingPadRole.json"
with open( config_filename , "r") as f:
    config_doc = json.loads(f.read())

shared_acct.put_role_and_policy_config ( config_doc )